In [2]:
import os
from yt_dlp import YoutubeDL
import cv2
import imutils
import requests
import gc
import time
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from vidgear.gears import CamGear
from imutils.object_detection import non_max_suppression
from imutils import paths

2023-08-21 13:37:28.764090: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-21 13:37:28.873741: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-21 13:37:29.680534: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
os.environ['TF_GPU_ALLOCATOR']  = 'cuda_malloc_async'
print(tf.__version__)  # 2.13.0-rc2
print(tf.config.list_physical_devices('GPU')) # [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
print(tf.test.is_built_with_cuda)  # <function is_built_with_cuda at 0x169361d00>
print(tf.test.gpu_device_name())  # /device:GPU:0
print(tf.config.get_visible_devices())  # [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

2.13.0
[]
<function is_built_with_cuda at 0x7fdf84d50550>

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


2023-08-21 13:37:30.645278: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-08-21 13:37:30.645306: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: ITZ
2023-08-21 13:37:30.645309: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: ITZ
2023-08-21 13:37:30.645412: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.86.5
2023-08-21 13:37:30.645424: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.86.5
2023-08-21 13:37:30.645426: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.86.5


In [4]:
# Download model if not already downloaded
if not os.path.isdir('mobilenet_v2_140_224'):
    if not os.path.isfile('mobilenet_v2_140_224.1.zip'):
        !wget https://github.com/GantMan/nsfw_model/releases/download/1.2.0/mobilenet_v2_140_224.1.zip
        !unzip mobilenet_v2_140_224.1.zip

In [5]:
# Load model
model = tf.keras.models.load_model('mobilenet_v2_140_224')
# Test model
model(np.zeros((64, 224, 224, 3)))  # array([[0.001, 0.999]], dtype=float32)

<tf.Tensor: shape=(64, 5), dtype=float32, numpy=
array([[0.28783095, 0.20058176, 0.49027213, 0.01533243, 0.00598276],
       [0.28783095, 0.20058176, 0.49027213, 0.01533243, 0.00598276],
       [0.28783095, 0.20058176, 0.49027213, 0.01533243, 0.00598276],
       [0.28783095, 0.20058176, 0.49027213, 0.01533243, 0.00598276],
       [0.28783095, 0.20058176, 0.49027213, 0.01533243, 0.00598276],
       [0.28783095, 0.20058176, 0.49027213, 0.01533243, 0.00598276],
       [0.28783095, 0.20058176, 0.49027213, 0.01533243, 0.00598276],
       [0.28783095, 0.20058176, 0.49027213, 0.01533243, 0.00598276],
       [0.28783095, 0.20058176, 0.49027213, 0.01533243, 0.00598276],
       [0.28783095, 0.20058176, 0.49027213, 0.01533243, 0.00598276],
       [0.28783095, 0.20058176, 0.49027213, 0.01533243, 0.00598276],
       [0.28783095, 0.20058176, 0.49027213, 0.01533243, 0.00598276],
       [0.28783095, 0.20058176, 0.49027213, 0.01533243, 0.00598276],
       [0.28783095, 0.20058176, 0.49027213, 0.01533243

In [6]:
# Get labels
labels = []
with open('mobilenet_v2_140_224/class_labels.txt', 'r') as f:
    labels = [l.strip() for l in f.readlines()]
for l in labels:
    print(l)


output = model(tf.random.uniform([1, 224, 224, 3]))
print(output)

drawings
hentai
neutral
porn
sexy
tf.Tensor([[7.0968449e-02 1.3347461e-02 9.1523635e-01 3.0026623e-04 1.4752209e-04]], shape=(1, 5), dtype=float32)


In [7]:
# initialize the HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

In [8]:
# Find GStreamer version
buildInfo = cv2.getBuildInformation()
SUPPORT_GSTREAMER = True
for line in buildInfo.split('\n'):
    if 'GStreamer' in line:
        print(line)
        if "NO" in line:
            SUPPORT_GSTREAMER = False
        

    GStreamer:                   NO


In [9]:
youtube_video_id = "thJgU9jkdU4"
url = "https://youtu.be/" + youtube_video_id
source = None
filename_ydl = None
if not SUPPORT_GSTREAMER:
    # Download instead of streaming
    if "youtu" in url:
        with YoutubeDL() as ydl:
            ydl.download([url])
            info = ydl.extract_info(url, download=False)
            # Get file path
            filename_ydl = ydl.prepare_filename(info)
            print(filename_ydl)
    else:
        # Download video
        filename_ydl = url.split('/')[-1]
        if not os.path.isfile(filename_ydl):
            resp = requests.get(url, stream=True)
            with open(filename_ydl, 'wb') as f:
                for chunk in resp.iter_content(chunk_size=1024):
                    f.write(chunk)
    
    

[youtube] Extracting URL: https://youtu.be/thJgU9jkdU4
[youtube] thJgU9jkdU4: Downloading webpage
[youtube] thJgU9jkdU4: Downloading ios player API JSON
[youtube] thJgU9jkdU4: Downloading android player API JSON
[youtube] thJgU9jkdU4: Downloading m3u8 information
[info] thJgU9jkdU4: Downloading 1 format(s): 616+251
[download] Rammstein - Dicke Titten (Official Video) [thJgU9jkdU4].webm has already been downloaded
[youtube] Extracting URL: https://youtu.be/thJgU9jkdU4
[youtube] thJgU9jkdU4: Downloading webpage
[youtube] thJgU9jkdU4: Downloading ios player API JSON
[youtube] thJgU9jkdU4: Downloading android player API JSON
[youtube] thJgU9jkdU4: Downloading m3u8 information
Rammstein - Dicke Titten (Official Video) [thJgU9jkdU4].webm


In [10]:
labelsToColor = {
    "drawings": (0, 0, 0),
    "hentai": (0, 0, 255),
    "neutral": (0, 0, 0),
    "porn": (0, 255, 0),
    "sexy": (255, 0, 0)
}

triggerOnLabels = ["porn", "sexy", "hentai"]

directory = (filename_ydl or url).split('/')[-1].split('.')[0]

os.makedirs(directory, exist_ok=True)

In [12]:
# infinite loop
if not SUPPORT_GSTREAMER:
    print("Using file: " + filename_ydl)
    source = CamGear(source=filename_ydl, logging=True)
else:
    source = CamGear(source=url, stream_mode = True, logging=True)

DISABLE_CHUNKING = True

stream = source.start() # YouTube Video URL as input
# Print resolution of stream
frame = stream.read()
if frame is None: 
    raise Exception("Unable to read stream from source")
print(frame.shape)
# How many 224x224 square you can fit in the frame
squareXCount = frame.shape[1] // 224
squareYCount = frame.shape[0] // 224
print("Square X Count: " + str(squareXCount))
print("Square Y Count: " + str(squareYCount))

# Offset to center the squares based on leftover pixels
startOffsetX = (squareXCount * 224)
startOffsetY = (squareYCount * 224)

# How many pixels are left over
leftoverX = frame.shape[1] - startOffsetX
leftoverY = frame.shape[0] - startOffsetY

print("Start Offset X: " + str(startOffsetX))
print("Start Offset Y: " + str(startOffsetY))

offsetX = int(leftoverX  / 2)
offsetY = int(leftoverY / 2) 

print("Offset X: " + str(offsetX))
print("Offset Y: " + str(offsetY))

fps = stream.framerate
print("FPS: " + str(fps))
last_triggred_frame = 0
frame_count = 0
last_frame_time = time.time()
while True:
    if frame_count % 100 == 0:
        # Prevent TensorFlow memory leak
        gc.collect()
    frame = stream.read()
    # read frames

    # check if frame is None
    if frame is None:
        #if True break the infinite loop
        break
    frame_count += 1
    # do something with frame here
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # detect people in the image
    #(rects, weights) = hog.detectMultiScale(gray, winStride=(4, 4), padding=(8, 8), scale=1.05)
    #pick = non_max_suppression(rects, probs=None, overlapThresh=0.65)
    
    #for (x, y, w, h) in pick:
    #    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
    # Divide frame into 224x224 squares by drawing rectangles
    # For each square, predict if it is NSFW
    # If NSFW, blur the square
    batchOfBatch = []
    for i in range(frame.shape[0] // 224):
        if DISABLE_CHUNKING: break
        batch = []
        for j in range(frame.shape[1] // 224):
            square_points = [(j * 224 + offsetX, i * 224 + offsetY), (j * 224 + 224 + offsetX, i * 224 + 224 + offsetY)]
            # Crop square
            square = frame[square_points[0][1]:square_points[1][1], square_points[0][0]:square_points[1][0]]
            
            # Write square to file
            #cv2.imwrite("square.jpg", square)
            # Resize square to 224x224 
            square = cv2.resize(square, (224, 224))
            # Convert square to RGB
            square = cv2.cvtColor(square, cv2.COLOR_BGR2RGB)
            # Convert square to float32
            square = square.astype(np.float32)
            # Normalize square
            square /= 255
            # Add square to batch
            batchOfBatch.append(square)
            #print("Shape of square: " + str(square.shape))
            
    original_frame = frame.copy()
    og = cv2.resize(frame, (224, 224))
    og = cv2.cvtColor(og, cv2.COLOR_BGR2RGB)
    og = og.astype(np.float32)
    og /= 255
    batchOfBatch.append(og)
    batch = np.array(batchOfBatch)
    predictions = model.predict(batch, verbose=None, batch_size=64)
    triggered = False
    indexBatch = 0
    for i in range(frame.shape[0] // 224):
        if DISABLE_CHUNKING: break
        # Predict NSFW for batch
        # Blur NSFW squares
        for j in range(frame.shape[1] // 224):
            prediction = predictions[indexBatch]
            square_points = [(j * 224 + offsetX, i * 224 + offsetY), (j * 224 + 224 + offsetX, i * 224 + 224 + offsetY)]
            # Write prediction on frame
            for k in range(5):
                if prediction[k] > 0.7 and labels[k] in triggerOnLabels:
                    #triggered = True
                    frame[i * 224 + offsetY:i * 224 + 224 + offsetY, j * 224 + offsetX:j * 224 + 224 + offsetX] = cv2.blur(frame[i * 224 + offsetY:i * 224 + 224 + offsetY, j * 224 + offsetX:j * 224 + 224 + offsetX], (50, 50))
                text = labels[k] + ": " + str(round(prediction[k], 2))
                color = labelsToColor[labels[k]]
                cv2.rectangle(frame, (j * 224 + offsetX, i * 224 + offsetY + 20 * k), (j * 224 + offsetX + int(prediction[k] * 100), i * 224 + offsetY + 20 * (k + 1)), color, -1)
                cv2.putText(frame, text, (j * 224 + offsetX, i * 224 + offsetY + 25 * k), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

                
            cv2.rectangle(frame, square_points[0], square_points[1], (0, 0, 255), 2)
            
            indexBatch += 1
    original_prediction = predictions[-1]
    main_trigger_reason = ""
    for k in range(5):
        text = labels[k] + ": " + str(round(original_prediction[k], 2))
        color = labelsToColor[labels[k]]
        cv2.rectangle(frame, (0, 20 * (k+1)), (int(original_prediction[k] * 100), 20 * (k + 1)), color, -1)
        cv2.putText(frame, text, (0, 25 * (k+1)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        if original_prediction[k] > 0.65 and labels[k] in triggerOnLabels:
            triggered = True
            main_trigger_reason = main_trigger_reason + "[" + labels[k] + " " + str(round(original_prediction[k], 2)) + "]"
    

    
    
    if triggered:
        cv2.putText(frame, "NSFW DETECTED", (0, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        # Save original frame if more than fps * n frames have passed since last trigger
        if frame_count - last_triggred_frame > fps * 0.25:
            cv2.imwrite(directory + "/" + str(frame_count) + main_trigger_reason + ".jpg", original_frame)
            last_triggred_frame = frame_count
    
    # Get FPS measurement
    current_time = time.time()
    fps = 1 / (current_time - last_frame_time)
    last_frame_time = current_time
    cv2.putText(frame, "FPS: " + str(round(fps, 2)), (int(frame.shape[1]/2), 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow("Output Frame", frame)
    # Show output window

    key = cv2.waitKey(1) & 0xFF
    # check for 'q' key-press
    if key == ord("q"):
        #if 'q' key-pressed break out
        break

cv2.destroyAllWindows()
# close output window

# safely close video stream.
stream.stop()

13:45:05 ::    CamGear    ::  DEBUG   :: Enabling Threaded Queue Mode for the current video source!


Using file: Rammstein - Dicke Titten (Official Video) [thJgU9jkdU4].webm
(1080, 1920, 3)
Square X Count: 8
Square Y Count: 4
Start Offset X: 1792
Start Offset Y: 896
Offset X: 64
Offset Y: 92
FPS: 25.0


13:53:12 ::    CamGear    ::  DEBUG   :: Terminating processes.
